# Apache Spark's DataFrame Usage

In previous tutorials, we have seen the popularity of [Pandas](https://github.com/EQWorks/python-curriculum/blob/main/basics/03-data-containers-and-repetitions-3.md) as well as the [application of its functions](https://github.com/EQWorks/python-curriculum/blob/main/data-analysis/04-aggregation-and-visualization.ipynb). In this notebook, we will look at Apache Spark's library [`pyspark`](https://spark.apache.org/docs/latest/api/python/) with the scope of comparing basic uses through examples.

In [1]:
import pyspark
from pyspark import SparkContext, SparkFiles
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

In [2]:
sc = SparkContext()
spark = SparkSession(sc)

## Loading & Viewing Data

To load a dataset from a remote URL we can utilize `SparkContext.addFile()` and `SparkFiles.get()`. These functions allow for `pyspark` to resolve the absolute path of the data. Whereas in pandas we would simply put the URL into the file path of `.read_csv()` to be resolved.

When comparing the display of a pandas dataframe to a `pyspark` dataframe, the difference is very apparent. 
There is a quick fix for displaying the data in a neater format by utilizing `.toPandas()`

In [3]:
url = 'https://raw.githubusercontent.com/EQWorks/python-curriculum/main/data/sample_boards_report.csv'
spark.sparkContext.addFile(url)

In [4]:
# Truncate is set to False, because we would like to see full column values
spark.read.option("header",True).csv(SparkFiles.get("sample_boards_report.csv")).show(1, False)
#.limit(1).toPandas()

+--------+-------+------------+----------------------------------------+--------+---------+------+-----+----------+--------------+------------------------+--------------------+--------+-----------+--------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+-------+--------+--------+--------+--------+-------+--------+--------+--------+--------+-------+--------+-------+--------+--------+--------+-----+-----+--------+-----+-----+--------+--------+-----+-----+--------+-----+-----+-----+-----+--------+-----+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-------+--------+
|board_id|market |product_name|sales_address                           |lat     |long     |height|width|facing    |total_observed|audience_target_observed|audience_composition|index   |day_of_week|AD_00  

In [5]:
pd.read_csv(url).head(1)

,board_id,market,product_name,sales_address,lat,long,height,width,facing,total_observed,...,I_14,I_15,I_16,I_17,I_18,I_19,I_20,I_21,I_22,I_23
0,7387802,Toronto,Static,HUMBER COLLEGE - MF-J-Block J107-M-WAL 1,43.72806,-79.60669,17.0,13.0,Main Floor,49,...,0.79739,0.653214,0.531563,0.616949,0.652551,0.685661,0.605912,0.471603,0.74584,0.407592


Another way to view the data is by looking at the schema. In Pandas we use `.dtypes()`) to show us the data type of each column.

Unless the schema is specified while reading the file, the default for all columns will `string`. We can either rewrite the schema independently or add infer schema option when reading file

In [6]:
spark.read.option("header",True).csv(SparkFiles.get("sample_boards_report.csv")).printSchema() 

root
 |-- board_id: string (nullable = true)
 |-- market: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- sales_address: string (nullable = true)
 |-- lat: string (nullable = true)
 |-- long: string (nullable = true)
 |-- height: string (nullable = true)
 |-- width: string (nullable = true)
 |-- facing: string (nullable = true)
 |-- total_observed: string (nullable = true)
 |-- audience_target_observed: string (nullable = true)
 |-- audience_composition: string (nullable = true)
 |-- index: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- AD_00: string (nullable = true)
 |-- AD_01: string (nullable = true)
 |-- AD_02: string (nullable = true)
 |-- AD_03: string (nullable = true)
 |-- AD_04: string (nullable = true)
 |-- AD_05: string (nullable = true)
 |-- AD_06: string (nullable = true)
 |-- AD_07: string (nullable = true)
 |-- AD_08: string (nullable = true)
 |-- AD_09: string (nullable = true)
 |-- AD_10: string (nullable = true)
 |-

In [7]:
dfSpark = spark.read.option("header",True).option("inferSchema", True).csv(SparkFiles.get("sample_boards_report.csv"))
# selecting columns for better viewing
dfSpark = dfSpark.select('board_id','market','day_of_week','product_name','total_observed','audience_target_observed', 'audience_composition', 'index')
dfSpark.printSchema()

root
 |-- board_id: integer (nullable = true)
 |-- market: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- total_observed: integer (nullable = true)
 |-- audience_target_observed: integer (nullable = true)
 |-- audience_composition: double (nullable = true)
 |-- index: double (nullable = true)



We have previously seen and heard of how the `Pandas` library offers many tools for data grouping, aggregation, reshaping, filtering, reformatting, etc. in tutorials such as [03-data-containers-and-repetitions-3](https://github.com/EQWorks/python-curriculum/blob/294b55343b469152463f920086bd2876fd282a21/basics/03-data-containers-and-repetitions-3.md) and [04-aggregation-and-visualization](https://github.com/EQWorks/python-curriculum/blob/main/data-analysis/04-aggregation-and-visualization.ipynb). In this section, we will continue exploring the offerings of the `pyspark` library to a similar extent, and how the library can be utilized interchangeably with Pandas to process data. We will also further examine the implementation of [SQL](https://github.com/EQWorks/python-curriculum/blob/main/data-analysis/11-work-with-sql.ipynb) in Spark. We will see a similar implementation of keyword use to process and examine data. 

## Filtering Data

As we continue with these examples, a strong resemblance to SQL-style coding will be prominent. 
```
    SELECT (product_name, total_observed, ...) FROM Table
```

In this first example we will use the keyword `select` to choose the columns we want displayed. 

In pandas, we would choose columns by implementing indexing with a column list e.g. `df[[column name list]]`


    

In [8]:
Filter = dfSpark.select('product_name','total_observed','audience_target_observed', 'audience_composition', 'index')
Filter.show(5, False)

+-----------------+--------------+------------------------+--------------------+--------+
|product_name     |total_observed|audience_target_observed|audience_composition|index   |
+-----------------+--------------+------------------------+--------------------+--------+
|Static           |49            |26                      |0.530612            |0.791408|
|Classique/Classic|63            |36                      |0.571429            |0.843304|
|Static           |54            |31                      |0.574074            |0.865937|
|Static           |183           |106                     |0.579235            |1.077622|
|Static           |48            |34                      |0.708333            |1.056479|
+-----------------+--------------+------------------------+--------------------+--------+
only showing top 5 rows



The equivalent of `.unique()` in pandas. 

Again we see keywords i.e. `select`, `distinct`, `collect`, similar to querying data in dataframe.

In [9]:
Filter.select('product_name').distinct().collect()

[Row(product_name='Bike Share'),
 Row(product_name='Transit Shelter'),
 Row(product_name='Horizontal'),
 Row(product_name='Classique/Classic'),
 Row(product_name='Static'),
 Row(product_name='Bike Share / Vélos en ville'),
 Row(product_name='Signature Column/Colonne Signature'),
 Row(product_name='Column')]

Filtering would be implemented similarily to Pandas by index `df[condition]`. We would use `&` `|` `~` to represent `AND`, `OR`, and `NOT`, instead of using the keyword as they would in a query. 

```
SELECT *
    FROM poi_lists
    WHERE name LIKE '%pizza%'
        OR name LIKE '%domino%'
```

In [10]:
Filter.filter((Filter.product_name == 'Static') & (Filter.total_observed == 49)).show(5,False)

+------------+--------------+------------------------+--------------------+--------+
|product_name|total_observed|audience_target_observed|audience_composition|index   |
+------------+--------------+------------------------+--------------------+--------+
|Static      |49            |26                      |0.530612            |0.791408|
+------------+--------------+------------------------+--------------------+--------+



## Creating New Columns

Keyword: `withColumn` 

Need to reference column names as `col(column name)`

In [17]:
Filter.withColumn('calculated_AC', round(col('audience_target_observed')/col('total_observed'), 6)).show(5, False)

+-----------------+--------------+------------------------+--------------------+--------+-------------+
|product_name     |total_observed|audience_target_observed|audience_composition|index   |calculated_AC|
+-----------------+--------------+------------------------+--------------------+--------+-------------+
|Static           |49            |26                      |0.530612            |0.791408|0.530612     |
|Classique/Classic|63            |36                      |0.571429            |0.843304|0.571429     |
|Static           |54            |31                      |0.574074            |0.865937|0.574074     |
|Static           |183           |106                     |0.579235            |1.077622|0.579235     |
|Static           |48            |34                      |0.708333            |1.056479|0.708333     |
+-----------------+--------------+------------------------+--------------------+--------+-------------+
only showing top 5 rows



## Merging DataFrames

As seen in [11-work-with-sql](https://github.com/EQWorks/python-curriculum/blob/main/data-analysis/11-work-with-sql.ipynb), the comparison between Pandas `merge()` function and SQL `joins` can achieve similar effects.

And do to its relation to using SQL functions, `join()` is the function to reproduce the effects of Pandas `merge()`.

In [18]:
Filter.join(dfSpark, on=['total_observed','audience_target_observed'], how='inner').show(5, False)
#.limit(5).toPandas()

+--------------+------------------------+-----------------+--------------------+--------+--------+-------+-----------+-----------------+--------------------+--------+
|total_observed|audience_target_observed|product_name     |audience_composition|index   |board_id|market |day_of_week|product_name     |audience_composition|index   |
+--------------+------------------------+-----------------+--------------------+--------+--------+-------+-----------+-----------------+--------------------+--------+
|49            |26                      |Static           |0.530612            |0.791408|7387802 |Toronto|Wed        |Static           |0.530612            |0.791408|
|63            |36                      |Classique/Classic|0.571429            |0.843304|7387802 |Toronto|Tue        |Classique/Classic|0.571429            |0.843304|
|54            |31                      |Static           |0.574074            |0.865937|7387802 |Toronto|Fri        |Static           |0.574074            |0.865937

## GroupBys

We see another concept from [11-work-with-sql](https://github.com/EQWorks/python-curriculum/blob/main/data-analysis/11-work-with-sql.ipynb), Groupbys. The concept of groupby is quite similar to that of Pandas `.groupby()`, even with the inclusion of aggregating multiple stats.

In [19]:
dfSpark.groupBy("product_name").sum("index").show(truncate=False)

+----------------------------------+-----------------+
|product_name                      |sum(index)       |
+----------------------------------+-----------------+
|Bike Share                        |5.392304         |
|Transit Shelter                   |75.97666600000001|
|Horizontal                        |1.28497          |
|Classique/Classic                 |0.843304         |
|Static                            |6.441741         |
|Bike Share / Vélos en ville       |1.316308         |
|Signature Column/Colonne Signature|1.245631         |
|Column                            |1.580411         |
+----------------------------------+-----------------+



In [20]:
dfSpark.groupBy("product_name").agg(sum("index").alias('index_total'),
                                    avg("audience_composition").alias('avg_ac')).show(truncate=False)


+----------------------------------+-----------------+------------------+
|product_name                      |index_total      |avg_ac            |
+----------------------------------+-----------------+------------------+
|Bike Share                        |5.392304         |0.8032934999999999|
|Transit Shelter                   |75.97666600000001|0.8458088070175439|
|Horizontal                        |1.28497          |0.854839          |
|Classique/Classic                 |0.843304         |0.571429          |
|Static                            |6.441741         |0.6619383333333334|
|Bike Share / Vélos en ville       |1.316308         |0.896552          |
|Signature Column/Colonne Signature|1.245631         |0.842466          |
|Column                            |1.580411         |0.84949           |
+----------------------------------+-----------------+------------------+



## Creating user defined functions

We can create functions as we would normally `def function(dependents): ...`

In [15]:
def avg_ac(a, t):
    
    return a/t

# Registers function to allow user to preset return schema
spark.udf.register("Avg_AC", avg_ac, LongType())

<function __main__.avg_ac(a, t)>

In [21]:
Filter.withColumn("function_test", avg_ac(col('audience_target_observed'),col('total_observed')).alias("calc_ac_test")).show(5, False)

+-----------------+--------------+------------------------+--------------------+--------+------------------+
|product_name     |total_observed|audience_target_observed|audience_composition|index   |function_test     |
+-----------------+--------------+------------------------+--------------------+--------+------------------+
|Static           |49            |26                      |0.530612            |0.791408|0.5306122448979592|
|Classique/Classic|63            |36                      |0.571429            |0.843304|0.5714285714285714|
|Static           |54            |31                      |0.574074            |0.865937|0.5740740740740741|
|Static           |183           |106                     |0.579235            |1.077622|0.5792349726775956|
|Static           |48            |34                      |0.708333            |1.056479|0.7083333333333334|
+-----------------+--------------+------------------------+--------------------+--------+------------------+
only showing top 5 